## Paso 2 – Modelo CNN Aumentado

En este apartado se construye un **modelo convolucional más complejo** que parte de la arquitectura base definida en el paso anterior, añadiendo varias capas para mejorar su capacidad de aprendizaje y generalización.

### Características del modelo:

- Se han añadido **tres bloques convolucionales**, cada uno compuesto por:
  - Una capa `Conv2D` con filtros crecientes (32 → 64 → 128).
  - Una capa `BatchNormalization` para estabilizar y acelerar el entrenamiento.
  - Una capa `MaxPooling2D` para reducir dimensionalidad y controlar el sobreajuste.
  
- Después del bloque convolucional, se utiliza:
  - Una capa `Flatten` para convertir las características 2D en un vector 1D.
  - Una capa `Dense` con 128 neuronas y activación `ReLU` como capa oculta densa.
  - Una capa `Dropout` del 50% para regularizar y evitar sobreajuste.
  - Una capa de salida `Dense(6)` con activación `softmax`, adaptada a la clasificación multiclase.

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from utils.dataloader import load_data_npy, DataGenerator
from utils.model_utils import save_model_and_history

2025-05-27 14:10:05.460600: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 14:10:05.871310: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 14:10:06.329036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748355006.662735   49465 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748355006.761009   49465 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748355008.196940   49465 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Cargar los datos
images_train, categories_train, images_val, categories_val, images_test, categories_test = load_data_npy()

# Crear generadores
train_gen = DataGenerator(images_train, categories_train)
val_gen = DataGenerator(images_val, categories_val, shuffle=False)
test_gen = DataGenerator(images_test, categories_test, shuffle=False)

print(f"Train: {images_train.shape}, Validation: {images_val.shape}, Test: {images_test.shape}")

Train: (10220, 150, 150, 3), Validation: (2555, 150, 150, 3), Test: (4259, 150, 150, 3)


In [3]:
# Modelo CNN extendido
model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=30,
                    callbacks=[early_stop])

# Evaluación final
test_loss, test_acc = model.evaluate(test_gen)
print(f"\nTest Accuracy: {test_acc:.4f}  |  Test Loss: {test_loss:.4f}")

# Guardado
save_model_and_history(
    model,
    history,
    model_path='model_extended_cnn'
)

2025-05-27 14:15:53.932633: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 486s 1s/step - accuracy: 0.4251 - loss: 4.2237 - val_accuracy: 0.3014 - val_loss: 2.5081
Epoch 2/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 524s 2s/step - accuracy: 0.5099 - loss: 1.2531 - val_accuracy: 0.3675 - val_loss: 1.4076
Epoch 3/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 511s 2s/step - accuracy: 0.5544 - loss: 1.1240 - val_accuracy: 0.6544 - val_loss: 1.4072
Epoch 4/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 476s 1s/step - accuracy: 0.6096 - loss: 1.0374 - val_accuracy: 0.6932 - val_loss: 1.0169
Epoch 5/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 469s 1s/step - accuracy: 0.6392 - loss: 0.9284 - val_accuracy: 0.7018 - val_loss: 0.8148
Epoch 6/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 437s 1s/step - accuracy: 0.6668 - loss: 0.8722 - val_accuracy: 0.5123 - val_loss: 1.6312
Epoch 7/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 414s 1s/step - accuracy: 0.6771 - loss: 0.8441 - val_accuracy: 0.7260 - val_loss: 0.7615
Epoch 8/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 434s 1s/step - accuracy: 0.7072 - loss: 0.7316 - val_accu


Test Accuracy: 0.8140  |  Test Loss: 0.5570
Modelo guardado en: /opt/notebooks/M9/models/model_extended_cnn.h5
Historial guardado en: /opt/notebooks/M9/models/model_extended_cnn_history.json
